# Plots for Thesis Paper

In [ ]:
import util
import math
import numpy as np

from blurs import *

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go

## Plots of PSF Functions

In [ ]:
psf_linear = linear((32, 32), *[7, math.pi/4])
psf_gaussian = gaussian((32, 32), *[3, 1.5])
psf_radial = radial((32, 32), 9)

from plotly import subplots
from scipy.fft import ifft2, fftshift

psf_linear = util.normalize(np.real(fftshift(ifft2(psf_linear))))
psf_gaussian = util.normalize(np.real(fftshift(ifft2(psf_gaussian))))
psf_radial = util.normalize(np.real(fftshift(ifft2(psf_radial))))
psfs = [psf_linear, psf_radial, psf_gaussian]

fig = subplots.make_subplots(rows=1, cols=3)
i = 1
for p in psfs:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_linear, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=1000, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

# fig = px.imshow(psf_linear, aspect='equal', color_continuous_scale='gray')
# fig.update_layout(width=400, height=400, margin=dict(l=10, r=10, b=10, t=10))
# fig.show()

# fig = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray')
# fig.update_layout(width=400, height=400, margin=dict(l=10, r=10, b=10, t=10))
# fig.show()

# fig = px.imshow(psf_radial, aspect='equal', color_continuous_scale='gray')
# fig.update_layout(width=400, height=400, margin=dict(l=10, r=10, b=10, t=10))
# fig.show()

# Shapes Blur

In [ ]:
image = util.load_img('../samples/shapes.png')

psf_params = [2, 2]
psf_fn = gaussian
psf = psf_fn(image.shape, *psf_params)

from scipy.fft import ifft2, fftshift
psf_img = np.real(fftshift(ifft2(psf)))
psf_gaussian = util.normalize(np.real(fftshift(ifft2(psf))))

image_blurred = blur(image, psf)

# optional: add noise
noise_factor = 0
image_blurred = noise(image_blurred, noise_factor)

images = [util.normalize(image), image_blurred]

fig = subplots.make_subplots(rows=1, cols=2)
i = 1
for p in images:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=700, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

# Shapes Deblur

In [ ]:
image = util.load_img('../samples/shapes.png')

psf_params = [1, 1]
psf_fn = gaussian
psf = psf_fn(image.shape, *psf_params)

from scipy.fft import ifft2, fftshift
psf_img = np.real(fftshift(ifft2(psf)))
psf_gaussian = util.normalize(np.real(fftshift(ifft2(psf))))

image_blurred = blur(image, psf)

# optional: add noise
noise_factor = 0.005
image_blurred = noise(image_blurred, noise_factor)

def deblur_tik(alpha, S, blurred_img):
    # S is fft2(circshift(psf))

    # phi = abs(S).^2 ./ (abs(s).^2 + alpha^2)
    # perform adjustment to values which could result in division by zero
    denominator = np.absolute(np.square(S)) + np.square(alpha)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    phi = np.divide(np.absolute(np.square(S)), denominator)

    # filter out S values according to phi 
    S_filt = np.zeros(phi.shape)
    idx = S != 0
    S_filt[idx] = np.divide(phi[idx], S[idx])

    # deblur using filtered S
    X_filt = ifft2(np.multiply(fft2(blurred_img), S_filt)).real

    return X_filt

image_deblurred = deblur_tik(0, psf, image_blurred)

images = [image_blurred, util.normalize(image_deblurred)]

fig = subplots.make_subplots(rows=1, cols=2)
i = 1
for p in images:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=700, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

# Deblur with Incorrect PSF

In [ ]:
image = util.load_img('../samples/shapes.png')

psf_params = [1, 1]
psf_fn = gaussian
psf = psf_fn(image.shape, *psf_params)

from scipy.fft import ifft2, fftshift
psf_img = np.real(fftshift(ifft2(psf)))
psf_gaussian = util.normalize(np.real(fftshift(ifft2(psf))))

image_blurred = blur(image, psf)

# optional: add noise
noise_factor = 0.0
image_blurred = noise(image_blurred, noise_factor)
image_deblurred = deblur_tik(0, psf, image_blurred)

new_psf = linear(image.shape, *[3.5, 0])
deblur_new_psf = deblur_tik(0, new_psf, image_blurred)

images = [image_blurred, util.normalize(image_deblurred), util.normalize(deblur_new_psf)]

fig = subplots.make_subplots(rows=1, cols=3)
i = 1
for p in images:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=1000, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

# Regularization

In [ ]:
import numpy as np
from scipy.optimize import minimize

def deblur_tik(alpha, S, blurred_img):
    # S is fft2(circshift(psf))

    # phi = abs(S).^2 ./ (abs(s).^2 + alpha^2)
    # perform adjustment to values which could result in division by zero
    denominator = np.absolute(np.square(S)) + np.square(alpha)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    phi = np.divide(np.absolute(np.square(S)), denominator)

    # filter out S values according to phi 
    S_filt = np.zeros(phi.shape)
    idx = S != 0
    S_filt[idx] = np.divide(phi[idx], S[idx])

    # deblur using filtered S
    X_filt = ifft2(np.multiply(fft2(blurred_img), S_filt)).real

    return X_filt

def deblur_tik_min(S):
    # error function; norm of deblurred image with blurred image
    error_function = lambda input, S, blurred_img: np.linalg.norm(deblur_tik(input[0], S, blurred_img) - image, ord='fro')

    initial_alpha = 0.1
    # since minimizer takes in a vector, need to get first element (which will store alpha)
    result = minimize(error_function, [initial_alpha], args=(S, image_blurred), bounds=[(0, 32)])

    # returns min alpha
    return result

def deblur_psf(psf_params, deblurred_img):
    S_i = linear(image.shape, *psf_params)
    return blur(deblurred_img, S_i)

def error_function(psf_params, deblurred_img, blurred_img):
    result = np.linalg.norm(deblur_psf(psf_params, deblurred_img) - blurred_img, ord='fro')
    return result

def deblur_psf_min(deblurred_img, initial_length):
    # min_{length, angle} = ||A(length, angle) x - b||
    # error_function = lambda psf_params, deblurred_img, blurred_img: np.linalg.norm(deblur_psf(psf_params, deblurred_img) - blurred_img, ord='fro')

    # specifting the solver as 'Powell' was important; otherwise it was stuck at wrong values
    result = minimize(error_function, [initial_length], args=(deblurred_img, image_blurred), method='Powell', bounds=[(0, 16)])

    return result

alphas = [0, 0.05, 0.3]
image_sequence = [util.normalize(deblur_tik(alpha, psf, image_blurred)) for alpha in alphas]

fig = subplots.make_subplots(rows=1, cols=3)
i = 1
for p in image_sequence:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(psf_gaussian, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=1000, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

# Alternating Minimizer

In [ ]:
prev_li = math.nan
li      = 10
prev_ai = math.nan
ai      = 0

images = []

step = 1
# while not math.isclose(prev_li, li) and not math.isclose(prev_ai, ai):
for i in range(20):
    # fix length and angle for guess
    psfi = linear(image.shape, li, ai)
    min_alpha = deblur_tik_min(psfi).x[0]
    deblurred_img = deblur_tik(min_alpha, psfi, image_blurred)

    # fix image for guess
    next_A = deblur_psf_min(deblurred_img, li)
    prev_li = li
    prev_ai = ai
    li = next_A.x[0]

    images.append(np.real(deblurred_img))
    step += 1

fig = px.imshow(np.array(images), animation_frame=0, color_continuous_scale='gray', title='Deblurred Image', width=512, height=512,
                labels=dict(animation_frame="iteration"))
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()

# Galaxy Images

In [ ]:
image_before = util.load_img('../samples/galaxy_before.png')[:,:,0]
image_after = util.load_img('../samples/galaxy_after.png')[:,:,0]

images = [image_before, image_after]

fig = subplots.make_subplots(rows=1, cols=2)
i = 1
for p in images:
    fig.add_trace(px.imshow(p).data[0], row=1, col=i)
    i += 1

layout = px.imshow(image_before, aspect='equal', color_continuous_scale='gray').layout
fig.layout.coloraxis = layout.coloraxis
fig.update_layout(width=700, height=300, margin=dict(l=10, r=10, b=10, t=10))
fig.show()

In [ ]:
def deblur_tik(alpha, S, blurred_img):
    # S is fft2(circshift(psf))

    # phi = abs(S).^2 ./ (abs(s).^2 + alpha^2)
    # perform adjustment to values which could result in division by zero
    denominator = np.absolute(np.square(S)) + np.square(alpha)
    denominator = np.where(denominator == 0, np.finfo(float).eps, denominator)
    phi = np.divide(np.absolute(np.square(S)), denominator)

    # filter out S values according to phi 
    S_filt = np.zeros(phi.shape)
    idx = S != 0
    S_filt[idx] = np.divide(phi[idx], S[idx])

    # deblur using filtered S
    X_filt = ifft2(np.multiply(fft2(blurred_img), S_filt)).real

    return X_filt

def deblur_tik_min(S):
    # error function; norm of deblurred image with blurred image
    error_function = lambda input, S, blurred_img: np.linalg.norm(deblur_tik(input[0], S, blurred_img) - image, ord='fro')

    initial_alpha = 0.1
    # since minimizer takes in a vector, need to get first element (which will store alpha)
    result = minimize(error_function, [initial_alpha], args=(S, image_blurred), bounds=[(0, 32)])

    # returns min alpha
    return result

def deblur_psf(psf_params, deblurred_img):
    S_i = gaussian(image.shape, psf_params[0], psf_params[0])
    return blur(deblurred_img, S_i)

def error_function(psf_params, deblurred_img, blurred_img):
    result = np.linalg.norm(deblur_psf(psf_params, deblurred_img) - blurred_img, ord='fro')
    return result

def deblur_psf_min(deblurred_img, initial_length):
    # min_{length, angle} = ||A(length, angle) x - b||
    # error_function = lambda psf_params, deblurred_img, blurred_img: np.linalg.norm(deblur_psf(psf_params, deblurred_img) - blurred_img, ord='fro')

    # specifting the solver as 'Powell' was important; otherwise it was stuck at wrong values
    result = minimize(error_function, [initial_length], args=(deblurred_img, image_blurred), method='Powell', bounds=[(0, 16)])

    return result

In [ ]:
image = image_after

psf_params = [5, 5]
psf_fn = gaussian
psf = psf_fn(image.shape, *psf_params)

image_blurred = blur(image, psf)

# optional: add noise
noise_factor = 0.02
image_blurred = noise(image_blurred, noise_factor)

prev_si = math.nan
si      = 0.5

images = []

step = 1
while not math.isclose(prev_si, si) and step < 10:
# for i in range(20):
    # fix length and angle for guess
    psfi = gaussian(image.shape, si, si)
    min_alpha = deblur_tik_min(psfi).x[0]
    deblurred_img = deblur_tik(min_alpha, psfi, image_blurred)

    # fix image for guess
    next_A = deblur_psf_min(deblurred_img, si)
    prev_si = si
    si = next_A.x[0]

    images.append(np.real(deblurred_img))
    step += 1

fig = px.imshow(np.array(images), animation_frame=0, color_continuous_scale='gray', title='Deblurred Image', width=512, height=512,
                labels=dict(animation_frame="iteration"))
fig.update_layout(coloraxis_showscale=False)
fig.update_xaxes(showticklabels=False)
fig.update_yaxes(showticklabels=False)
fig.show()